In [ ]:
%websocketconnect 192.168.1.130:8266 --password horsetoe -kbi 

In [ ]:
2+2

In [ ]:
global wdt_counter
wdt_counter = 0

![Pinout](nodemcu_pins.png)

![Pinout2](ESP8266-NodeMCU-kit-12-E-pinout-gpio-pin.png)

![Pin details](pin_details.png)

![Schematic](schematic.png)

### Photon pin - ESP8266 pin

D2 - D6 (Machine pin 12)  is rain gauge

D3 - D5 (Machine pin 14) is wind speed

A0 - A0 is wind vane

D1 - D1 / SCL
D0 - D2 / SDA

# Set up pins

## LED

In [ ]:
import machine

In [ ]:
frequency = 5000

In [ ]:
led = machine.PWM(machine.Pin(2), frequency)

In [ ]:
led.duty(0)

In [ ]:
led.duty(1023)

## Wind direction

#### Wind direction is hooked up to the analog to digital coverter on pin 0

In [ ]:
wind_direction_pin = machine.ADC(0)

In [ ]:
wind_direction_pin.read()

#### Translate the voltage read from the ADC (range 0-1024) to a cardinal direction

![Wind Vane Resistors](wind_vane_resistors.png)

Manually checked each direction, and these are the raw values I get

N = 825

NNW = 741

NW = 925

WNW = 867

W = 985

WSW = 634

SW = 666

SSW = 268

S = 314

SSE = 144

SE = 207

ESE = 77

E = 108

ENE = 97

NE = 495

NNE = 439

In [ ]:
vals = [825, 741, 925, 867, 985, 634, 666, 268, 314, 144, 207, 77, 108, 97, 495, 439]

In [ ]:
vals.sort()

In [ ]:
vals

In [ ]:
wind_direction_pin.read()

In [ ]:
wind_direction_dict = {(0, 60):    "Error.low",
                       (61, 85):   "ESE",
                       (86, 103):  "ENE",
                       (104, 130): "E",
                       (131, 180): "SSE",
                       (181, 240): "SE",
                       (241, 290): "SSW",
                       (291, 400): "S",
                       (401, 470): "NNE",
                       (471, 560): "NE",
                       (561, 650): "WSW",
                       (651, 700): "SW",
                       (701, 780): "NNW",
                       (781, 845): "N",
                       (846, 900): "WNW",
                       (901, 950): "NW",
                       (951, 1000):"W",
                       (1001, 1024):"Error.high"}

In [ ]:
def getDirection(table, raw):
    for key in table:
        if key[0] <= raw <= key[1]:
            return table[key]

In [ ]:
wind_direction_raw = wind_direction_pin.read()

In [ ]:
wind_direction_raw

In [ ]:
getDirection(wind_direction_dict, wind_direction_raw)

## Wind speed

Interrupts may be attached to any GPIO pin except GPIO16. Since GPIO6-GPIO11 are typically used to interface with the flash memory ICs on most esp8266 modules, applying interrupts to these pins are likely to cause problems

Following https://people.eecs.berkeley.edu/~boser/courses/49_sp_2019/N_gpio.html#_interrupts and https://github.com/DL1CB/RamsbergWeatherStation

In [ ]:
anemometer_count = 0

In [ ]:
DEBOUNCE_MS = const(50)

class pulseCount:

    def __init__(self, pin, callback=None, falling=True):  
        """ Count anemometer switches with debouncing. Arguments:
        pin: configured pin (incl PULL_UP/DOWN)
        callback: handler, called when hall switch press detected
        falling: detect raising or falling edges
        """
        self.last_time_ms = 0
        self.detected = False  # a switch press was detected
        self.cb = callback
        pin.irq(self._irq_cb, pin.IRQ_FALLING if falling else pin.IRQ_RISING)

    def pressed(self):     
        """Return True if switch pressed since last call"""
        p = self.detected
        self.detected = False
        return p

    def _irq_cb(self, pin):    
        t = time.ticks_ms()
        diff = t - self.last_time_ms
        if abs(diff) < DEBOUNCE_MS:     
            return
        self.last_time_ms = t
        self.detected = True
        if self.cb: self.cb(pin)   

In [ ]:
def anemometer_reporter(pin):
    global anemometer_count
    anemometer_count = anemometer_count + 1

In [ ]:
anemometer_pin = machine.Pin(14, mode=machine.Pin.IN, pull=machine.Pin.PULL_UP)   
anemometer_detect = pulseCount(anemometer_pin, callback=anemometer_reporter) 

Per the data sheet "A wind speed of 2.4km/h causes the switch to close once per second." 

In [ ]:
def windSpeedKilometersPerHour(pulses, period): 
    """ kmh instantaneous wind speed """
    return round(2.4 * (pulses / period), 4)

## Rain meter

Interrupts may be attached to any GPIO pin except GPIO16. Since GPIO6-GPIO11 are typically used to interface with the flash memory ICs on most esp8266 modules, applying interrupts to these pins are likely to cause problems

Following https://people.eecs.berkeley.edu/~boser/courses/49_sp_2019/N_gpio.html#_interrupts and https://github.com/DL1CB/RamsbergWeatherStation

In [ ]:
rain_gauge_count = 0

In [ ]:
def rain_gauge_reporter(pin):
    global rain_gauge_count
    rain_gauge_count = rain_gauge_count + 1

In [ ]:
rain_gauge_pin = machine.Pin(12, mode=machine.Pin.IN, pull=machine.Pin.PULL_UP)   
rain_detect = pulseCount(rain_gauge_pin, callback=rain_gauge_reporter) 

The rain gauge is a self-emptying tipping bucket type. Each **0.2794mm** of rain causes one momentary contact closure.

In [ ]:
def readData():
    global anemometer_count
    global rain_gauge_count
    wind_direction_raw = wind_direction_pin.read()
    raw_weather_data = {"wind_direction_raw" : wind_direction_raw,
                        #"wind_direction_cardinal" : getDirection(wind_direction_dict, wind_direction_raw),
                        "wind_speed_raw" : anemometer_count,
                        #"wind_speed_kph" : windSpeedKilometersPerHour(anemometer_count, period),
                        "rain_amount_raw" : rain_gauge_count
                        #"rain_amount_ml" : countsToMilliliters(rain_gauge_count)
                       }
    #Reset the anemometer and rain gauge counts
    anemometer_count = 0
    rain_gauge_count = 0
    return raw_weather_data

In [ ]:
readData()

# Temp, barometric pressure, and humidity

In [ ]:
global wdt_counter
print(wdt_counter)
wdt_counter = 0

Hooked up to I2C (SDA and SCL)

SDA = machine pin 4 = D2

SCL = machine pin 5 = D1

In [ ]:
import machine

In [ ]:
i2c = machine.I2C(sda=machine.Pin(4), scl=machine.Pin(5))

Sample code at  https://github.com/chrisbalmer/micropython-si7021/blob/master/si7021.py

Data sheet at https://cdn.sparkfun.com/datasheets/Sensors/Weather/Si7021.pdf

`While the measurement is in progress, the option of either clock stretching (Hold Master Mode) or Not Acknowledging read requests (No Hold Master Mode) is available to indicate to the master that the measurement is in progress; the chosen command code determines which mode is used.`

#### Si7021 is at address 0x40 / 64

Measure Relative Humidity, Hold Master Mode 0xE5

Measure Relative Humidity, No Hold Master Mode 0xF5

Measure Temperature, Hold Master Mode 0xE3

Measure Temperature, No Hold Master Mode 0xF3

Read Temperature Value from Previous RH Measurement 0xE0

Reset 0xFE

Write RH/T User Register 1 0xE6

Read RH/T User Register 1 0xE7

Read Electronic ID 1st Byte 0xFA 0x0F

Read Electronic ID 2nd Byte 0xFC 0xC9

Read Firmware Revision 0x84 0xB8

In [ ]:
SI7021_i2c_address = 0x40

SI7021_measure_temperature = bytearray([0xF3])

SI7021_measure_relative_humidity = bytearray([0xF5])

SI7021_reset = bytearray([0xFE])

SI7021_read_ID_1 = bytearray([0xFA, 0x0F])

SI7021_read_ID_2 = bytearray([0xFC, 0xC9])

In [ ]:
def convert_bytes_to_integer(bytes_to_convert):
    'Use bitwise operators to convert the bytes into integers.'
    integer = None
    for chunk in bytes_to_convert:
        if not integer:
            integer = chunk
        else:
            integer = integer << 8
            integer = integer | chunk
    return integer

In [ ]:
def get_Si7021_device_info():
    '''Get the serial number and the sensor identifier. The identifier is
    part of the bytes returned for the serial number.
    '''
    # Serial 1st half
    i2c.writeto(SI7021_i2c_address, SI7021_read_ID_1)
    id1 = bytearray(8)
    utime.sleep(0.025)
    i2c.readfrom_into(SI7021_i2c_address, id1)

    # Serial 2nd half
    i2c.writeto(SI7021_i2c_address, SI7021_read_ID_2)
    id2 = bytearray(6)
    utime.sleep(0.025)
    i2c.readfrom_into(SI7021_i2c_address, id2)

    combined_id = bytearray([id1[0], id1[2], id1[4], id1[6],
                             id2[0], id2[1], id2[3], id2[4]])

    serial = convert_bytes_to_integer(combined_id)
    identifier = get_Si7021_device_identifier(id2[0])

    return serial, identifier


In [ ]:
def get_Si7021_device_identifier(identifier_byte):
        '''Convert the identifier byte to a device identifier. Values are based
        on the information from page 24 of the datasheet.
        '''
        if identifier_byte == 0x00 or identifier_byte == 0xFF:
            return 'engineering sample'
        elif identifier_byte == 0x0D:
            return 'Si7013'
        elif identifier_byte == 0x14:
            return 'Si7020'
        elif identifier_byte == 0x15:
            return 'Si7021'
        else:
            return 'unknown'

In [ ]:
import utime

In [ ]:
get_Si7021_device_info()

### Get temperature

In [ ]:
temperature_data = bytearray(3)

In [ ]:
i2c.writeto(SI7021_i2c_address, SI7021_measure_temperature)

In [ ]:
i2c.readfrom_into(SI7021_i2c_address, temperature_data)

In [ ]:
print(temperature_data)

In [ ]:
def verify_SI7021_checksum(data):
    crc = 0
    values = data[:2]
    checksum = int(data[-1])
    for value in values:
        crc = crc ^ value
        for _ in range(8, 0, -1):
            if crc & 0x80: #10000000
                crc <<= 1
                crc ^= 0x131 #100110001
            else:
                crc <<= 1
    if crc != checksum:
        return False
    else:
        return True

In [ ]:
verify_SI7021_checksum(temperature_data)

The third byte is a checksum, so only look at the first two for the temperature

In [ ]:
temperature_data_int = convert_bytes_to_integer(temperature_data[:2])

In [ ]:
print(temperature_data_int)

Temperature (C) =  ((175.72\*Temp_Code) / 65536) - 46.85

In [ ]:
((175.72*temperature_data_int) / 65536) - 46.85

In [ ]:
def get_SI7021_temperature_celsius(SI7021_i2c_address):
        'Retrieve data from the sensor and verify it with a CRC check.'
        data = bytearray(3)
        #Measure temperature command = 0xF3
        i2c.writeto(SI7021_i2c_address, bytearray([0xF3]))
        utime.sleep(0.02)

        i2c.readfrom_into(SI7021_i2c_address, data)
        value = convert_bytes_to_integer(data[:2])
        value = ((175.72*value) / 65536) - 46.85
        verified = verify_SI7021_checksum(data)
        if not verified:
            return False
        return value

In [ ]:
get_SI7021_temperature_celsius(SI7021_i2c_address)

%RH = (125\*RH_Code) / 65536 

In [ ]:
def get_SI7021_relative_humidity(SI7021_i2c_address):
        'Retrieve data from the sensor and verify it with a CRC check.'
        data = bytearray(3)
        #Measure relative humidity command = 0xF5
        i2c.writeto(SI7021_i2c_address, bytearray([0xF5]))
        utime.sleep(0.02)

        i2c.readfrom_into(SI7021_i2c_address, data)
        value = convert_bytes_to_integer(data[:2])
        value = ((125*value) / 65536)
        verified = verify_SI7021_checksum(data)
        if not verified:
            return False
        return value

In [ ]:
get_SI7021_relative_humidity(SI7021_i2c_address)

In [ ]:
global wdt_counter
print(wdt_counter)
wdt_counter = 0

# Pressure sensor

Uses MPL3115A2 sensor

data sheet at https://www.nxp.com/docs/en/data-sheet/MPL3115A2.pdf, code at https://github.com/PinsonJonas/MPL3115A2_MicroPython/blob/master/mpl3115a2.py and https://github.com/adafruit/Adafruit_CircuitPython_MPL3115A2/blob/main/adafruit_mpl3115a2.py

MPL3115A2_i2c_address = 0x60

    ALTITUDE = const(0)
    PRESSURE = const(1)

    MPL3115_I2CADDR = const(0x60)
    MPL3115_STATUS = const(0x00)
    MPL3115_PRESSURE_DATA_MSB = const(0x01)
    MPL3115_PRESSURE_DATA_CSB = const(0x02)
    MPL3115_PRESSURE_DATA_LSB = const(0x03)
    MPL3115_TEMP_DATA_MSB = const(0x04)
    MPL3115_TEMP_DATA_LSB = const(0x05)
    MPL3115_DR_STATUS = const(0x06)
    MPL3115_DELTA_DATA = const(0x07)
    MPL3115_WHO_AM_I = const(0x0c)
    MPL3115_FIFO_STATUS = const(0x0d)
    MPL3115_FIFO_DATA = const(0x0e)
    MPL3115_FIFO_SETUP = const(0x0e)
    MPL3115_TIME_DELAY = const(0x10)
    MPL3115_SYS_MODE = const(0x11)
    MPL3115_INT_SORCE = const(0x12)
    MPL3115_PT_DATA_CFG = const(0x13)
    MPL3115_BAR_IN_MSB = const(0x14)
    MPL3115_P_ARLARM_MSB = const(0x16)
    MPL3115_T_ARLARM = const(0x18)
    MPL3115_P_ARLARM_WND_MSB = const(0x19)
    MPL3115_T_ARLARM_WND = const(0x1b)
    MPL3115_P_MIN_DATA = const(0x1c)
    MPL3115_T_MIN_DATA = const(0x1f)
    MPL3115_P_MAX_DATA = const(0x21)
    MPL3115_T_MAX_DATA = const(0x24)
    MPL3115_CTRL_REG1 = const(0x26)
    MPL3115_CTRL_REG2 = const(0x27)
    MPL3115_CTRL_REG3 = const(0x28)
    MPL3115_CTRL_REG4 = const(0x29)
    MPL3115_CTRL_REG5 = const(0x2a)
    MPL3115_OFFSET_P = const(0x2b)
    MPL3115_OFFSET_T = const(0x2c)
    MPL3115_OFFSET_H = const(0x2d)

In [ ]:
MPL3115A2_i2c_address = 0x60
MPL3115A2_status = const(0x00)
MPL3115A2_control_register_1 = const(0x26) #CTRL_REG1 - control register 1 (address 26h) bit description
MPL3115A2_PT_data_configuration = const(0x13) #The PT_DATA_CFG register configures the pressure data, 
#temperature data and eventflag generator.

In [ ]:
# Set up barometer mode, not raw, oversampling 128, minimum time 512 ms
i2c.writeto_mem(MPL3115A2_i2c_address, MPL3115A2_control_register_1, bytes([0x38]))
i2c.writeto_mem(MPL3115A2_i2c_address, MPL3115A2_PT_data_configuration, bytes([0x07]))  # no events detected
self.i2c.writeto_mem(MPL3115A2_i2c_address, MPL3115A2_control_register_1, bytes([0x39]))  # active

In [ ]:
def read_MPL3115A2_status():\
    status = bytearray(1)
    while True:
        i2c.readfrom_mem_into(MPL3115A2_i2c_address, MPL3115A2_status, status)

        if(status[0] == 0):
            time.sleep(0.01)
            pass
        elif(status[0] & 0x04) == 4:
            return True
        else:
            return False

In [ ]:
def get_MPL3115A2_pressure(self):
        if self.mode == ALTITUDE:
            raise MPL3115A2exception("Incorrect Measurement Mode MPL3115A2")

        out_pressure = self.i2c.readfrom_mem(
            MPL3115_I2CADDR, MPL3115_PRESSURE_DATA_MSB, 3)

        pressure_int = (out_pressure[0] << 10) + (out_pressure[1] << 2) + ((out_pressure[2] >> 6) & 0x3)
        pressure_frac = (out_pressure[2] >> 4) & 0x03

        return float(pressure_int + pressure_frac / 4.0)

In [ ]:
global wdt_counter
print(wdt_counter)
wdt_counter = 0

In [ ]:
readData()


# All together

In [ ]:
import machine
import utime

wind_direction_pin = machine.ADC(0)

class pulseCount:

    DEBOUNCE_MS = const(50)
    def __init__(self, pin, callback=None, falling=True):  
        #Count anemometer switches with debouncing. Arguments:
        #pin: configured pin (incl PULL_UP/DOWN)
        #callback: handler, called when hall switch press detected
        #falling: detect raising or falling edges
        self.last_time_ms = 0
        self.detected = False  # a switch press was detected
        self.cb = callback
        pin.irq(self._irq_cb, pin.IRQ_FALLING if falling else pin.IRQ_RISING)

    def pressed(self):     
        #Return True if switch pressed since last call
        p = self.detected
        self.detected = False
        return p

    def _irq_cb(self, pin):    
        t = utime.ticks_ms()
        diff = t - self.last_time_ms
        if abs(diff) < DEBOUNCE_MS:     
            return
        self.last_time_ms = t
        self.detected = True
        if self.cb: self.cb(pin)  
        
anemometer_count = 0
anemometer_pin = machine.Pin(5, mode=machine.Pin.IN, pull=machine.Pin.PULL_UP)   

def anemometer_reporter(pin):
    global anemometer_count
    anemometer_count = anemometer_count + 1

anemometer_detect = pulseCount(anemometer_pin, callback=anemometer_reporter) 

rain_gauge_count = 0
rain_gauge_pin = machine.Pin(4, mode=machine.Pin.IN, pull=machine.Pin.PULL_UP)   

def rain_gauge_reporter(pin):
    global rain_gauge_count
    rain_gauge_count = rain_gauge_count + 1

rain_detect = pulseCount(rain_gauge_pin, callback=rain_gauge_reporter) 

reset_time = utime.ticks_ms()

def readData():
    global anemometer_count
    global rain_gauge_count
    global reset_time
    wind_direction_raw = wind_direction_pin.read()
    raw_weather_data = {"wind_direction_raw" : wind_direction_raw,
                        #"wind_direction_cardinal" : getDirection(wind_direction_dict, wind_direction_raw),
                        "wind_speed_raw" : anemometer_count,
                        #"wind_speed_kph" : windSpeedKilometersPerHour(anemometer_count, period),
                        "rain_amount_raw" : rain_gauge_count,
                        #"rain_amount_ml" : countsToMilliliters(rain_gauge_count)
                        "timestamp" : list(utime.localtime()),
                        "elapsed_time" : utime.ticks_diff(utime.ticks_ms(), reset_time)
                       }
    #Reset the anemometer and rain gauge counts
    anemometer_count = 0
    rain_gauge_count = 0
    reset_time = utime.ticks_ms()
    return raw_weather_data

In [ ]:
readData()

In [ ]:
from collections.deque import deque

data_queue = deque()

data_queue.append(readData())
data_queue.popleft()

In [ ]:
sensors_script = """import machine
import utime

wind_direction_pin = machine.ADC(0)

class pulseCount:

    DEBOUNCE_MS = const(50)
    def __init__(self, pin, callback=None, falling=True):  
        #Count anemometer switches with debouncing. Arguments:
        #pin: configured pin (incl PULL_UP/DOWN)
        #callback: handler, called when hall switch press detected
        #falling: detect raising or falling edges
        self.last_time_ms = 0
        self.detected = False  # a switch press was detected
        self.cb = callback
        pin.irq(self._irq_cb, pin.IRQ_FALLING if falling else pin.IRQ_RISING)

    def pressed(self):     
        #Return True if switch pressed since last call
        p = self.detected
        self.detected = False
        return p

    def _irq_cb(self, pin):    
        t = utime.ticks_ms()
        diff = t - self.last_time_ms
        if abs(diff) < DEBOUNCE_MS:     
            return
        self.last_time_ms = t
        self.detected = True
        if self.cb: self.cb(pin)  
        
anemometer_count = 0
anemometer_pin = machine.Pin(14, mode=machine.Pin.IN, pull=machine.Pin.PULL_UP)   

def anemometer_reporter(pin):
    global anemometer_count
    anemometer_count = anemometer_count + 1

anemometer_detect = pulseCount(anemometer_pin, callback=anemometer_reporter) 

rain_gauge_count = 0
rain_gauge_pin = machine.Pin(12, mode=machine.Pin.IN, pull=machine.Pin.PULL_UP)   

def rain_gauge_reporter(pin):
    global rain_gauge_count
    rain_gauge_count = rain_gauge_count + 1

rain_detect = pulseCount(rain_gauge_pin, callback=rain_gauge_reporter) 

reset_time = utime.ticks_ms()

def readData():
    global anemometer_count
    global rain_gauge_count
    global reset_time
    wind_direction_raw = wind_direction_pin.read()
    raw_weather_data = {"wind_direction_raw" : wind_direction_raw,
                        #"wind_direction_cardinal" : getDirection(wind_direction_dict, wind_direction_raw),
                        "wind_speed_raw" : anemometer_count,
                        #"wind_speed_kph" : windSpeedKilometersPerHour(anemometer_count, period),
                        "rain_amount_raw" : rain_gauge_count,
                        #"rain_amount_ml" : countsToMilliliters(rain_gauge_count)
                        "timestamp" : utime.localtime(),
                        "elapsed_time" : utime.ticks_diff(utime.ticks_ms(), reset_time)
                       }
    #Reset the anemometer and rain gauge counts
    anemometer_count = 0
    rain_gauge_count = 0
    reset_time = utime.ticks_ms()
    return raw_weather_data
"""

In [ ]:
import os
print(os.listdir())

In [ ]:
with open('sensors.py') as f:
    print(f.read())

In [ ]:
with open('sensors.txt', 'w') as f:
    f.write(sensors_script)

### Double check the write

In [ ]:
with open('sensors.txt') as f:
    print(f.read())

## Move temp file to `boot.py`

In [ ]:
os.rename('sensors.txt', 'sensors.py')

In [ ]:
%disconnect